In [1]:
# Import tools
from splinter import Browser
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Create empyt DataFrame
AF1 = pd.DataFrame(columns = ['ID', 'name', 'continent', 'country', 'region', 'af_type', 'latitude', 'longitude', 'elevation',  'runway_L', 'runway_W', 'freq_count', 'airline_count'])
print(AF1)

Empty DataFrame
Columns: [ID, name, continent, country, region, af_type, latitude, longitude, elevation, runway_L, runway_W, freq_count, airline_count]
Index: []


In [3]:
# Create the temporary dataframe
temp_df = pd.DataFrame(columns =['ID', 'name', 'continent', 'country', 'region', 'af_type', 'latitude', 'longitude', 'elevation', 'runway_L', 'runway_W', 'freq_count', 'airline_count'])
temp_df

,ID,name,continent,country,region,af_type,latitude,longitude,elevation,runway_L,runway_W,freq_count,airline_count


In [4]:
# Import URL data
url_data_df = "Resources/Pacific_airfield_complete.csv"

In [5]:
# create URL dataframe
url_surface = pd.read_csv(url_data_df)
url_surface

,url,surface
0,https://airportdatabase.net/australia/badu-isl...,2
1,https://airportdatabase.net/australia/horn-isl...,3
2,https://airportdatabase.net/australia/kubin-ai...,2
3,https://airportdatabase.net/norfolk-island/nor...,3
4,https://airportdatabase.net/fiji/cicia-airport...,1
...,...,...
438,https://airportdatabase.net/vanuatu/tongoa-isl...,1
439,https://airportdatabase.net/vanuatu/torres-air...,1
440,https://airportdatabase.net/vanuatu/ulei-airpo...,1
441,https://airportdatabase.net/vanuatu/valesdir-a...,1


In [6]:
# Create list of runway surface data
surface_df = (url_surface['surface'])
print(surface_df)

0      2
1      3
2      2
3      3
4      1
      ..
438    1
439    1
440    1
441    1
442    1
Name: surface, Length: 443, dtype: int64


In [7]:
# Create URL for HTML table
for index, row in url_surface.iterrows():
    url = row['url']
   
    # Read HTML table
    df1 = pd.read_html(url)[0]

    # Rename columns for ease of reading
    df1.columns = ['Col_1', 'Col_2']

    # Create Name variable
    df1.loc[0, 'Col_1'] = 'name'
    df1['Col_2'] = df1['Col_2'].str.replace(' details and information', '')
    
    # Clean text; remove "ft."
    df1['Col_2'] = df1['Col_2'].str.replace(' ft.', '')

    # Clean text; remove colons
    df1['Col_1'] = df1['Col_1'].str.replace(':', '')

    # Clean text; remove "iso"
    df1['Col_1'] = df1['Col_1'].str.replace('iso ', '')

    # Clean text; remove "ISO"
    df1['Col_1'] = df1['Col_1'].str.replace('ISO ', '')

    # Clean text; remove " Code"
    df1['Col_1'] = df1['Col_1'].str.replace(' Code', '_Code')

    # Remove null row
    df1 = df1.fillna(0)

    # create name variable
    name = df1.loc[df1['Col_1'].str.contains('name', na=False)]
    name2 = name.iloc[0]['Col_2']
    
    # create ID variable
    ID = df1.loc[df1['Col_1'].str.contains('ident', na=False)]
    ID2 = ID.iloc[0]['Col_2']

    # create latitude variable
    latitude = df1.loc[df1['Col_1'].str.contains('latitude', na=False)]
    latitude2 = latitude.iloc[0]['Col_2']

    # create type variable
    af_type = df1.loc[df1['Col_1'].str.contains('type', na=False)]
    af_type2 = af_type.iloc[0]['Col_2']

    # create longitude variable
    longitude = df1.loc[df1['Col_1'].str.contains('longitude', na=False)]
    longitude2 = longitude.iloc[0]['Col_2']

    # create elevation variable
    elevation = df1.loc[df1['Col_1'].str.contains('elevation', na=False)]
    elevation2 = elevation.iloc[0]['Col_2']

    # create continent variable
    continent = df1.loc[df1['Col_1'].str.contains('continent', na=False)]
    continent2 = continent.iloc[0]['Col_2']

    # create country variable
    country = df1.loc[df1['Col_1'].str.contains('country', na=False)]
    country2 = country.iloc[0]['Col_2']

    # create region variable
    region = df1.loc[df1['Col_1'].str.contains('Region', na=False)]
    region2 = region.iloc[0]['Col_2']

    # find runway data rows
    pattern = '[0-9]x[0-9]'
    runway = df1['Col_2'].str.contains(pattern, na=False)

    # tag runway data rows in table w/"True"
    runway_B = pd.concat([df1, runway], axis=1)

    # Assign column name to runway data tag
    runway_B.columns = ['Col_1', 'Col_2', 'Col_3']

    # Select runway data rows
    runway_C = runway_B.loc[runway_B['Col_3'] == True]
    if runway_C.shape[0] == 0:
        runway_L = 0
        runway_W = 0
    
    else:
    
        runway_C['length'] = ''
        runway_C['width'] = ''
   
        
    # parse runway length/width variables
        runway_C[['length', 'width']] = runway_C['Col_2'].str.split('x', expand=True)

    # create runway Dataframe
        runway_df = pd.DataFrame(runway_C, columns=['Col_1', 'Col_2', 'Col_3', 'length', 'width'])

    # Convert runway length/width to integer datatype
        runway_df['length'] = runway_df['length'].astype('int')
        runway_df['width'] = runway_df['width'].astype('int')

    # Create runway length/width totals
        runway_df.loc['runway_total']= runway_df.sum(numeric_only=True, axis=0)

    # Create total runway length variable
        runway_L = runway_df.loc['runway_total']['length']

    # Create total runway width variable
        runway_W = runway_df.loc['runway_total']['width']


    # count frequencies and navaids
    freq_df = df1.loc[df1['Col_2'].str.contains("Mhz", na=False)]

    # create freq_count variable
    freq_count = len(freq_df)
    
    # Identify airlines data row
    pattern = ' , '
    airlines = df1['Col_2'].str.contains(pattern, na=False)

    # tag airline data rows in table w/"True"
    airlines_B = pd.concat([df1, airlines], axis=1)

    # Assign column name to airline data tag
    airlines_B.columns = ['Col_1', 'Col_2', 'Col_3']

    # Select airline data row
    airlines_C = airlines_B.loc[airlines_B['Col_3'] == True]

    # Create airlines Dataframe
    airlines_df = pd.DataFrame(airlines_C, columns=['Col_1', 'Col_2', 'Col_3'])

    # Parse airlines data into rows
    airlines_df2 = (airlines_df.set_index(['Col_3'])
       .apply(lambda x: x.str.split(',').explode())
       .reset_index())

    # Tag row with null data
    airlines_df2['Col_1'].replace('', np.nan, inplace=True)

    # Delete row with null data
    airlines_df2.dropna(subset=['Col_1'], inplace=True)

    # Create airline_count variable
    airline_count = len(airlines_df2)

    # Write data to temp Dataframe
    temp_df.at[0, 'ID'] = ID2
    temp_df.at[0, 'name'] = name2
    temp_df.at[0, 'continent'] = continent2
    temp_df.at[0, 'country'] = country2
    temp_df.at[0, 'region'] = region2
    temp_df.at[0, 'af_type'] = af_type2
    temp_df.at[0, 'latitude'] = latitude2
    temp_df.at[0, 'longitude'] = longitude2
    temp_df.at[0, 'elevation'] = elevation2
    temp_df.at[0, 'runway_L'] = runway_L
    temp_df.at[0, 'runway_W'] = runway_W
    temp_df.at[0, 'freq_count'] = freq_count
    temp_df.at[0, 'airline_count'] = airline_count

    # Convert temp Dataframe to correct data types
    temp_df = temp_df.astype({'latitude':'float', 'longitude':'float', 'elevation':'int', 'runway_L':'float', 'runway_W':'float', 'freq_count':'int', 'airline_count':'int'})

    # Append temp Dataframe to AF1
    AF1 = AF1.append(temp_df, ignore_index = True)



C:\Users\drchr\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  app.launch_new_instance()
C:\Users\drchr\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\drchr\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\drchr\anaconda3\envs\PythonData\lib\si

In [8]:
# Show AF1

AF1

,ID,name,continent,country,region,af_type,latitude,longitude,elevation,runway_L,runway_W,freq_count,airline_count
0,YBAU,Badu Island Airport,Oceania (OC),Australia (AU),Queensland (AU-QLD),small airport,-10.150000,142.173401,14,2788.0,0.0,4,0
1,YHID,Horn Island Airport,Oceania (OC),Australia (AU),Queensland (AU-QLD),medium airport,-10.586400,142.289993,43,8609.0,173.0,3,0
2,YKUB,Kubin Airport,Oceania (OC),Australia (AU),Queensland (AU-QLD),small airport,-10.225000,142.218002,15,3281.0,60.0,2,0
3,YSNF,Norfolk Island International Airport,Oceania (OC),Norfolk Island (NF),(unassigned) (NF-U-A),medium airport,-29.041599,167.938995,371,11106.0,248.0,4,4
4,NFCI,Cicia Airport,Oceania (OC),Fiji (FJ),Eastern (FJ-E),small airport,-17.743299,-179.341995,13,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,NVST,Tongoa Island Airport,Oceania (OC),Vanuatu (VU),Shefa (VU-SEE),small airport,-16.891100,168.550995,443,0.0,0.0,0,0
439,NVSD,Torres Airstrip,Oceania (OC),Vanuatu (VU),Torba (VU-TOB),small airport,-13.328000,166.638000,0,0.0,0.0,1,0
440,NVSU,Uléi Airport,Oceania (OC),Vanuatu (VU),Malampa (VU-MAP),small airport,-16.329700,168.301102,0,0.0,0.0,0,0
441,NVSV,Valesdir Airport,Oceania (OC),Vanuatu (VU),Shefa (VU-SEE),small airport,-16.796101,168.177002,10,0.0,0.0,0,0


In [9]:
# Save AF1 to AF3.csv

AF1.to_csv("Resources/AF3.csv", index=False)

In [ ]:
# Save Runway surface data to 